#### GISC 425 T1 2020
# **Assignment 3 Data processing using `dictionary` objects**

In [ ]:
# You need to run this cell to get things setup
%matplotlib inline

import matplotlib
import matplotlib.pyplot as pyplot

import geopandas

In this assignment you are asked to use a data dictionary to perform relatively complicated processing of a data set to make it more useful for mapping and analysis.

## Background information
The dataset we will be using is the Land Resource Inventory (LRI) which was digitized in the 1970s and includes extensive information on land use capability (or suitability), erosion,
geology, soils, and vegetation. LRI is a valuable resource for any work looking into changes in
land use over time. However, the form in which the attribute data for this database was recorded is a product of a time when computer memory and storage was at a premium. As a result very complex information about each map unit is recorded in a format that presents difficulties for present day users of the data interested in making even simple land capability, vegetation, soil or geological maps.

The particular data attribute we are interested in is Landuse Capability (LUC) which has been encoded in the format

    abcc+abcc
    
where the second part of this code may not always be present. The first four characters encode the dominant landuse capability assessment, and the second set of four (after the `+` sign) encode a subdominant capability where this has been assessed.

The characters in the assessment code are interpreted as explained below. First, the letter `a` denotes the Landuse Class as tabulated below

`a` | Landuse class
:--:|:-------------
1 2 3 or 4 | Arable
5 6 or 7 | Non-arable
8 | Protected

Next the letter `b` denotes the Landuse limitation

`b` | Landuse limitation
:--:|:------------------
c | Climate
e | Erosion
s | Soil
w | Wetness

Finally, the two characters `cc` are a two digit number ranking and associating various categories of land use suitability. Detailed interpretation of the number is beyond our scope in this assignment. Details can be found in:

+ Noble, K. E., 1985. *Land Use Capability Classification of the Southern Hawke’s Bay-Waiarapa
Region: A Bulletin to Accompany New Zealand Land Resource Inventory Worksheets*. Land
Resource Group Soil Conservation Centre, Aokautere, Ministry of Works and Development,
Palmerston North.

The four character `abcc` code may appear twice separated by a `+` character, in which
case the first set of four indicates the dominant land use capability assessment and the second
set a subdominant assessment.

Finally, 'non-normal' land units are coded as follows

Code | Meaning
:----|:-------
estu | Estuary
ice | Ice
lake | Lake
quar | Quarries/mines
rive | River
town | Town/urban


### So what's the problem?
So, for example the code

    3e 7+5e 5
    
decodes to

    Dominant class:          Arable
    Dominant limitation:     Erosion
    Dominant suitability:    7
    Subdominant class:       Non-arable
    Subdominant limitation:  Erosion
    Subdominant suitability: 5
    
That's a lot of information encoded in a single attribute, and you can maybe see why we would want to break it out into new variables in the dataset.

To see the problem even more clearly, we will read in a sample of the data (this is a national dataset, but it's useful to look at just a subset to get the idea).

In [ ]:
d = geopandas.read_file('wairarapa_s_hawkes_bay.geojson')

and

In [ ]:
d.plot(column='LUC', figsize=(8,11), linestyle='None')

Pretty map, but to get some idea of the problem, take a close look at the `LUC` column in the data table.

In [ ]:
d

Among the 6773 rows in this table, there are

In [ ]:
len(set(d['LUC']))

Yes... that's right, 105, unique codes! Although it is clear from the quick map we made above that there is useful spatial structure in these data, mapping and visulization is made difficult by so much information being summarised in a single (complex) encoding.

What we are aiming for is a way to unpack this information, per the tables in the previous section into  new variables each showing dominant and subdominant landuse class, landuse limitation, and landuse ranking. \[You can probably see that there's a similar problem lurking in the information contained in the `ROCK` and `SOIL` variables, but we'll leave that for some other time.]

In [ ]:
# Before continuing we'll read an even smaller dataset for testing
d = geopandas.read_file('sample_lri.geojson')

## An outline solution
There are probably a hundred different ways to approach this problem. I'm going to outline one approach, which you can follow. If you are feeling adventurous (and confident) feel free to tackle it in a different way. This is how I would do it.

**First** write a function that takes as input a LUC code, and depending on the parameters, returns dominant or subdominant landuse class, landuse limitation or landuse suitability&mdash;that's six different possible outcomes. It will also be fine if we end up with six smaller simpler functions, each of them capable of decoding a single new variable.

**Second** ... well... that's kind of the whole plan! 

The question is how to execute it.

Here are some things to consider.

### Dictionaries as translators
We've seen in the book how a dictionary can be used to translate between two languages, or in our situation how it can be used to map keys in the encoded data to values that we want in our output data. For example, if we had a dictionary with

    dictionary['town'] = 'Town/urban'
    
then it can do a small bit of the translation we need.  Similarly, provided we know that we are trying to determine the landuse class, a dictionary that decodes an input `1`, `2`, `3`, or `4` as `Arable` would also be useful. Same goes for one that can handle the landuse limitation. The suitability is a bit different. Since those are two digit numeric codes it's fine just to retain them as 

So... I suggest that the first thing to do is build a dictionary that can do most of the decoding for us. Start work on this in the cell below.

### **Build your translator `dict` here**

In [ ]:
## Setup a dictionary to do the hard work of the decoding
## It's OK if it can only handle some cases to begin 
## It's also OK to make two or three simple dictionaries 
## rather than one big complicated one
## Look at the tables in the Background information section 
## back at the top of this notebook for inspiration

## I've made a start on one of them for you below
non_normals = {'estu': 'Estuary'}

### Adding new variables to a `GeoDataFrame`
This is pretty easy, as it turns out. We simply do

    d['NewVariableName'] = ...
    
where we obviously need to fill in the `...` with some result from a function.

This is where `geopandas` is a big help. Instead of looping through every value in the `LUC` column of the dataset if we have some function (say) `decode_landuse_class` then we can do this:

    d.LUC.apply(decode_landuse_class)
    
to apply it to the `LUC` codes.  So in this case

    d['DOM_LUCLASS`] = d.LUC.apply(decode_landuse_class)
    
would do the trick. That suggests we need some functions...

### Simple functions for each of the six possible decodings
We want to decode the `LUC` codes to six different new variables, so at least to begin, this suggests making six (simple) functions that can take a `LUC` code and convert it to a value appropriate for whichever of the six new variables we want. So for example, you might have a function `decode_dom_class(code)` and another `decode_dom_limitation(code)` and so on.
    
There is likely to be scope for combining functions given that the dominant and subdominant assessment codes are the same (each is made up of four characters encoded identically, just before and after the `+` sign). But this is a refinement. To begin with, writing six simple functions that handle just one of the decoding operations will be perfectly fine.

Again, I've made a start in the cell below.

### **Build functions to do the conversions here**

In [ ]:
# function to return the dominant landuse class
# given a LUC code
def dom_landuse(code):
    if code in non_normals:
        return non_normals[code]
    else:
        return lu_classes[code[0]] # this won't work yet, because 'lu_classes' dict does not exist

## You'll need to write functions for the other new variables also 

### Making it work
The cell below shows how this would work for one variable

### **Add to the code below to do all the needed conversions**

In [ ]:
d['DOM_LUCLASS'] = d.LUC.apply(dom_landuse) # This won't work because work is needed above

And here is how to plot it to check your handiwork.

In [ ]:
fig = pyplot.figure(figsize=(8,11))

ax1 = fig.add_subplot(111)
d.plot(ax=ax1, column='DOM_LUCLASS', cmap='tab20b')

## Submission instructions
Once you have all the parts working, in the various cells above, double check everything by resetting the notebook and running everything from a newly reopened copy.

Then put everything (dictionary, functions, commands to run the functions) in the single cell below.

Also, this time, **add comments to your code**. This is to help me assess how well you understand what you are doing, and will be an important part of the grading.

Finally, save the notebook, ensuring your name is included in the filename, and submit via the dropbox provided on Blackboard. The assignment is due on **10 April at 5:00PM**.

In [ ]:
## Assemble all the code needed to do the processing in this cell
## Make sure to comment your code, so I can tell what you think you are doing
%matplotlib inline

import matplotlib
import matplotlib.pyplot as pyplot

import geopandas

## --------------------------------------------------------------------------
## Initiatlise the dictionary here:
## --------------------------------------------------------------------------


## --------------------------------------------------------------------------
## Provide function definitions here:
## --------------------------------------------------------------------------



## --------------------------------------------------------------------------
## And now run the code
## --------------------------------------------------------------------------

# Start by reading the data
d = geopandas.read_file('wairarapa_s_hawkes_bay.geojson')

# Over to you!
#
#